# Monarch

One of Monarch's more powerful features is its Actor/endpoint API, which provides a generic interface for distributed computing. In this notebook, we introduce some of the basics

## Hello World
Actors are spawned in Process meshes via the `monarch.proc_mesh` API. For those familiar with distributed systems, it can be helpful to think of each Actor as a server with endpoints that can be called.

In [2]:
import asyncio

from monarch.proc_mesh import proc_mesh, ProcMesh
from monarch.actor_mesh import Actor, endpoint, current_rank

NUM_ACTORS=4

class ToyActor(Actor):
    def __init__(self):
        self.rank = current_rank().rank
    
    @endpoint
    async def hello_world(self, msg):
        print(f"Identity: {self.rank}, {msg=}")        

# Note: Meshes can be also be created on different nodes, but we're ignoring that in this example
local_proc_mesh = await proc_mesh(gpus=NUM_ACTORS)
# This spawns 4 instances of 'ToyActor'
toy_actor = await local_proc_mesh.spawn("toy_actor", ToyActor) 

I0529 17:04:38.195836 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _128tEcETi5FK[0] rank 0: created
I0529 17:04:38.196293 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _128tEcETi5FK[1] rank 1: created
I0529 17:04:38.196709 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _128tEcETi5FK[2] rank 2: created
I0529 17:04:38.197093 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _128tEcETi5FK[3] rank 3: created
I0529 17:04:39.233129 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:133] proc _128tEcETi5FK[1] rank 1: running at addr:unix!@choXYEwnmQspRu9iZUgKvF6Q mesh_agent:_128tEcETi5FK[1].mesh[0]<hyperactor_mesh::proc_mesh::mesh_agent::MeshAgent>
I0529 17:04:39.234619 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:133] proc _128tEcETi5FK[3] rank 3: running at addr:unix!@aztljBHYamc5fQB8oKyUptrm mesh_agent:_128tEcETi5FK[3].mesh[0]<hyperactor_mesh::proc_mesh::mesh_agent::MeshAgent>
I0529 17:04:39.238444 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:133] proc _128tEcETi5

In [5]:
# Once actors are spawned, we can call all of them simultaneously with `Actor.endpoint.call` as below
await toy_actor.hello_world.call("hey there, from jupyter!!")

Identity: 3, msg='hey there, from jupyter!!'
Identity: 2, msg='hey there, from jupyter!!'
Identity: 0, msg='hey there, from jupyter!!'
Identity: 1, msg='hey there, from jupyter!!'


In [6]:
# We can also specify a single actor using the 'slice' API
futures = []
for idx in range(NUM_ACTORS):
    actor_instance = toy_actor.slice(gpus=idx)
    futures.append(actor_instance.hello_world.call_one(f"Here's an arbitrary unique value: {idx}"))

# conveniently, we can still schedule & gather them in parallel using asyncio
await asyncio.gather(*futures)

Identity: 0, msg="Here's an arbitrary unique value: 0"
Identity: 0, msg="Here's an arbitrary unique value: 1"
Identity: 0, msg="Here's an arbitrary unique value: 2"
Identity: 0, msg="Here's an arbitrary unique value: 3"


## Ping Pong
Not only is it possible to call endpoints froma 'main' fuction, but actors have the useful property of being able to communicate with one another. 

In [20]:
import asyncio

from monarch.proc_mesh import proc_mesh, ProcMesh
from monarch.actor_mesh import Actor, endpoint, current_rank

class ExampleActor(Actor):
    def __init__(self, actor_name):
        self.actor_name=actor_name
    
    @endpoint
    async def init(self, other_actor):
        self.other_actor = other_actor
        self.other_actor_pair = other_actor.slice(**current_rank())
        self.identity = current_rank().rank
    
    @endpoint
    async def send(self, msg):
         await self.other_actor_pair.recv.call(f"Sender ({self.actor_name}:{self.identity}) {msg=}")
        
    @endpoint
    async def recv(self, msg):
        print(f"Pong!, Receiver ({self.actor_name}:{self.identity}) received msg {msg}")

# Spawn two different Actors in different meshes, with two instances each
local_mesh_0 = await proc_mesh(gpus=2)
actor_0 = await local_mesh_0.spawn(
    "actor_0",
    ExampleActor,
    "actor_0"     # this arg is passed to ExampleActor.__init__
) 

local_mesh_1 = await proc_mesh(gpus=2)
actor_1 = await local_mesh_1.spawn(
    "actor_1",
    ExampleActor,
    "actor_1"     # this arg is passed to ExampleActor.__init__
) 

I0529 17:05:37.904476 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _14wENoic4WQM[0] rank 0: created
I0529 17:05:37.904983 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _14wENoic4WQM[1] rank 1: created
I0529 17:05:39.065343 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:133] proc _14wENoic4WQM[0] rank 0: running at addr:unix!@tjxHFPUc2tv5w8RYZDOjVTFN mesh_agent:_14wENoic4WQM[0].mesh[0]<hyperactor_mesh::proc_mesh::mesh_agent::MeshAgent>
I0529 17:05:39.070276 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:133] proc _14wENoic4WQM[1] rank 1: running at addr:unix!@yHbPlBV7H43hbCq73DsCoPTQ mesh_agent:_14wENoic4WQM[1].mesh[0]<hyperactor_mesh::proc_mesh::mesh_agent::MeshAgent>
I0529 17:05:39.289522 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _1xtVcyVB4hVH[0] rank 0: created
I0529 17:05:39.290061 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:114] proc _1xtVcyVB4hVH[1] rank 1: created
I0529 17:05:40.331259 2830017 hyperactor_mesh/src/proc_mesh/mod.rs:133] proc _1xtVcyVB4h

In [25]:
# Initialize each actor with references to each other
await asyncio.gather(
    actor_0.init.call(actor_1),
    actor_1.init.call(actor_0),
)

In [26]:
await actor_0.send.call("Ping")

Pong!, Receiver (actor_1:0) received msg Sender (actor_0:0) msg='Ping'
Pong!, Receiver (actor_1:1) received msg Sender (actor_0:1) msg='Ping'


In [27]:
await actor_1.send.call("Ping")

Pong!, Receiver (actor_0:0) received msg Sender (actor_1:0) msg='Ping'
Pong!, Receiver (actor_0:1) received msg Sender (actor_1:1) msg='Ping'
